# 添加消息历史记录

RunnableWithMessageHistory 允许我们将消息历史记录添加到某些类型的链中。它包装另一个 Runnable 并管理它的聊天消息历史记录。

具体来说，它可用于任何将以下之一作为输入的 Runnable

- 一个字典，其键采用 BaseMessage 序列
- 一个字典，其键将最新消息作为 BaseMessage 的字符串或序列，以及一个单独的键，将历史消息作为字符串或序列

并作为输出之一返回

- 可以视为 AIMessage 内容的字符串
- 一个字典，其键包含 BaseMessage 序列

让我们看一些示例，看看它是如何工作的。首先我们构造一个可运行的程序（这里接受一个字典作为输入并返回一条消息作为输出）：

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("LOACL_API_KEY")
api_base = os.getenv("LOACL_API_BASE")

model = ChatOpenAI(api_key=api_key, base_url=api_base, temperature=0.3)


prompt = ChatPromptTemplate.from_messages(
    [
        ('system', '你是一个擅长{ability}的专家， 用20个字或更少的字数进行简洁概要的回复。'),
        MessagesPlaceholder(variable_name='history'),
        ('user', '{input}'),
    ]
)

runnable = prompt | model


为了管理消息历史记录，我们需要： 
1. 这个可运行程序； 
2. 返回 BaseChatMessageHistory 实例的可调用函数。

查看内存集成页面，了解使用 Redis 和其他提供程序实现聊天消息历史记录。在这里，我们演示如何使用内存中 ChatMessageHistory 以及使用 RedisChatMessageHistory 进行更持久的存储。

## In-memory 内存中

下面我们展示了一个简单的示例，其中聊天历史记录位于内存中，在本例中通过全局 Python 字典。

我们构造一个可调用的 get_session_history ，它引用此字典以返回 ChatMessageHistory 的实例。可以通过在运行时将配置传递给 RunnableWithMessageHistory 来指定可调用的参数。默认情况下，配置参数应为单个字符串 session_id 。这可以通过 history_factory_config kwarg 进行调整。

使用单参数默认值：

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(runnable, get_session_history=get_session_history, input_messages_key='input', history_messages_key='history')

请注意，我们指定了 input_messages_key （被视为最新输入消息的键）和 history_messages_key （添加历史消息的键）。

当调用这个新的可运行时，我们通过配置参数指定相应的聊天历史记录：

In [8]:
with_message_history.invoke(
    {"ability": "数学", "input": "余弦是什么意思？"},
    config={"configurable": {"session_id": "def234"}},
)

AIMessage(content='余弦是直角三角形中邻边与斜边的比值，或单位圆上的横坐标。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 32, 'total_tokens': 68, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-4d7c06a2-d5c7-4135-9d9d-d16baa50b872-0', usage_metadata={'input_tokens': 32, 'output_tokens': 36, 'total_tokens': 68, 'input_token_details': {}, 'output_token_details': {}})

In [9]:
with_message_history.invoke(
    {"ability": "math", "input": "能否再说一遍？"},
    config={"configurable": {"session_id": "def234"}},
)

AIMessage(content='余弦是直角三角形中邻边与斜边的比值，或单位圆上角的横坐标。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 67, 'total_tokens': 104, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5e936010-1ec6-4dea-a99e-92b70071f7ae-0', usage_metadata={'input_tokens': 67, 'output_tokens': 37, 'total_tokens': 104, 'input_token_details': {}, 'output_token_details': {}})

In [10]:
with_message_history.invoke(
    {"ability": "math", "input": "能否再说一遍？"},
    config={"configurable": {"session_id": "ABC123"}},
)

AIMessage(content='可以，请问具体是指哪部分内容？', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 34, 'total_tokens': 47, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c7d30a75-f8c0-4ea3-9c9c-00c2fbe1045e-0', usage_metadata={'input_tokens': 34, 'output_tokens': 13, 'total_tokens': 47, 'input_token_details': {}, 'output_token_details': {}})

我们可以通过将 ConfigurableFieldSpec 对象列表传递给 history_factory_config 参数来自定义用于跟踪消息历史记录的配置参数。下面，我们使用两个参数： user_id 和 conversation_id 。

In [12]:
from langchain_core.runnables import ConfigurableFieldSpec


store = {}

def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]

with_message_history = RunnableWithMessageHistory(
    runnable, 
    get_session_history=get_session_history, 
    input_messages_key='input',
    history_messages_key='history',
    history_factory_config=[
        ConfigurableFieldSpec(
            id='user_id', 
            name='User ID', 
            annotation=str,
            description='用户的唯一标识符',
            default='',
            is_shared=True
        ), 
        ConfigurableFieldSpec(
            id='conversation_id', 
            name='Conversation ID', 
            annotation=str,
            description='对话的唯一标识符',
            default='',
            is_shared=True
        )
    ]
)


In [13]:
with_message_history.invoke(
    {"ability": "数学", "input": "余弦是什么意思？"},
    config={"configurable": {"user_id": "kkutys", "conversation_id": "kkutys_conv1"}},
)


AIMessage(content='余弦是角度的三角函数，表示直角三角形中邻边与斜边的比值。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 32, 'total_tokens': 63, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-82beed13-b3ba-4832-b1ba-3c781a6891c6-0', usage_metadata={'input_tokens': 32, 'output_tokens': 31, 'total_tokens': 63, 'input_token_details': {}, 'output_token_details': {}})

## 具有不同签名的可运行实例的示例

上面的可运行程序接受一个字典作为输入并返回一个 BaseMessage。下面我们展示了一些替代方案。

In [15]:
from langchain_core.messages import HumanMessage
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel({"output_message": model})

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history=get_session_history, 
    output_messages_key='output_message'
)

with_message_history.invoke(
    [HumanMessage(content="什么是二元一次方程？")],
    config={"configurable":{"session_id":"baz"}}
)

{'output_message': AIMessage(content='**二元一次方程**（也称为**二元一次方程组**的单个方程）是指含有两个变量的线性方程，其中每个变量的次数都是1。通常，这种方程的标准形式为：\n\n\\[\nAx + By = C\n\\]\n\n其中，**x** 和 **y** 是变量，**A**、**B** 和 **C** 是常数，且 **A** 和 **B** 不同时为零。\n\n### 二元一次方程的特点：\n1. **含有两个变量**：方程中有两个未知数，通常用 **x** 和 **y** 表示。\n2. **线性关系**：方程中的变量的指数都是1，意味着它表示的是一条直线。\n3. **常数与系数**：方程的右边是常数 **C**，左边包含两个项，分别由 **x** 和 **y** 与常数 **A** 和 **B** 相乘。\n\n### 例子：\n1. \\( 2x + 3y = 6 \\)\n2. \\( x - 4y = 7 \\)\n3. \\( 5x + 2y = 10 \\)\n\n这些方程都是二元一次方程，表示的是平面上的一条直线。\n\n### 解二元一次方程\n解二元一次方程的目的是找出 **x** 和 **y** 的具体值。由于二元一次方程通常有无数解（因为它表示的是一条直线），但如果给定了特定的条件（比如通过另一个方程），可以通过代数方法找到一个确定的解。\n\n#### 常见解法：\n1. **代入法**：通过解其中一个方程来表示一个变量（例如 \\(x\\)）为另一个变量的函数，代入另一个方程解得另一个变量（例如 \\(y\\)）。\n2. **消元法**：通过将两个方程相加或相减，消去其中一个变量，从而得到另一个变量的解。\n3. **图像法**：将方程转化为图像（直线方程），然后通过观察直线的交点来解得方程的解。\n\n### 举例说明解法：\n假设我们有以下方程组：\n\\[\n2x + 3y = 6\n\\]\n\\[\nx - 4y = 7\n\\]\n\n**代入法**：\n1. 从第二个方程 \\(x = 4y + 7\\) 解出 **x**。\n2. 将 **x** 代入第一个方程：\n   \\[\n   2(4y + 7) + 3y = 6\n   \\]\n   展开后得到：\

In [16]:
with_message_history.invoke(
    [HumanMessage(content="什么是未知数？")],
    config={"configurable":{"session_id":"baz"}}
)

{'output_message': AIMessage(content='**未知数**（也称为**变量**）是指在数学问题或方程中尚未确定、需要求解的数量。在方程或表达式中，未知数通常用字母表示，例如 **x**、**y**、**z** 等。\n\n### 1. **未知数的作用**：\n在数学中，未知数用来表示尚不清楚的数值。它们出现在代数方程、表达式、方程组、函数等中，通常是我们希望通过解方程来找出的数。换句话说，未知数是我们要求解的对象。\n\n例如：\n- 在方程 \\( 2x + 5 = 11 \\) 中，**x** 是未知数。我们需要通过解这个方程来找出 **x** 的值。\n- 在几何问题中，未知数可以表示某个边长、角度或其他数量。\n\n### 2. **未知数的符号**：\n在数学中，未知数通常使用字母来表示。常见的字母包括：\n- **x**, **y**, **z**：常用于表示代数中的未知数，尤其在一元或二元方程中。\n- **a**, **b**, **c**：常用于表示常数，或者在几何中用来表示特定的长度或坐标。\n- **t**, **s** 等：通常用于表示时间、速度、位置等动态量。\n\n### 3. **如何求解未知数**：\n通过给定的方程、条件或模型，未知数的值通常通过代数运算（如加、减、乘、除、开方等）来求解。常见的求解方法包括：\n- **代入法**：将一个方程中的某个未知数表示成另一个未知数的形式，并代入另一个方程进行求解。\n- **消元法**：通过对方程进行加减操作，消去一个未知数，最终求出另一个未知数的值。\n- **代数运算**：通过对方程进行直接运算，如移项、因式分解等，来求解未知数。\n\n例如：\n- 对于方程 \\( 3x - 5 = 10 \\)，可以通过移项求解：\n  \\[\n  3x = 10 + 5\n  \\]\n  \\[\n  3x = 15\n  \\]\n  \\[\n  x = 5\n  \\]\n  这时，**x** 就是未知数，解得 **x = 5**。\n\n### 4. **未知数与常数的区别**：\n- **常数**（如 2、5、7 等）是已经确定的数值。\n- **未知数** 是我们需要求解的数值，它在问题中通常没有明确的数值，需要通过计算、推理等方法得出。\n\n

## 持久存储

在许多情况下，最好保留对话历史记录。 RunnableWithMessageHistory 不知道 get_session_history 可调用对象如何检索其聊天消息历史记录。有关使用本地文件系统的示例，请参阅此处。下面我们演示如何使用 Redis。查看内存集成页面，了解使用其他提供程序实现聊天消息历史记录。

如果尚未安装 Redis，我们需要安装它：

pip install --upgrade --quiet redis

如果我们没有可连接的现有 Redis 部署，请启动本地 Redis Stack 服务器：

docker run -d -p 6379:6379 -p 8001:8001 redis/redis-stack:latest

REDIS_URL = "redis://localhost:6379/0"

更新消息历史记录实现只需要我们定义一个新的可调用对象，这次返回 RedisChatMessageHistory 的实例：


In [17]:
from langchain_community.chat_message_histories import RedisChatMessageHistory


REDIS_URL = "redis://localhost:6379/0"

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    return RedisChatMessageHistory(url=REDIS_URL, session_id=session_id)

with_message_history = RunnableWithMessageHistory(
    runnable,
    get_session_history=get_session_history,
    input_messages_key='input',
    history_messages_key='history'
)

with_message_history.invoke(
    {"ability": "数学", "input": "余弦是什么意思？"},
    config={"configurable":{"session_id":"def234"}}
)

AIMessage(content='余弦是三角函数，表示角度的直角三角形邻边与斜边的比值。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 32, 'total_tokens': 62, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b9f04980-0220-4685-856b-af628cc7dd31-0', usage_metadata={'input_tokens': 32, 'output_tokens': 30, 'total_tokens': 62, 'input_token_details': {}, 'output_token_details': {}})

In [18]:
with_message_history.invoke(
    {"ability": "math", "input": "What does cosine mean?"},
    config={"configurable": {"session_id": "foobar"}},
)

AIMessage(content='Cosine is a trigonometric function representing the ratio of adjacent side to hypotenuse in a right triangle.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 32, 'total_tokens': 55, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-90ad36b3-4807-41a2-8cf5-d9ffda0967d6-0', usage_metadata={'input_tokens': 32, 'output_tokens': 23, 'total_tokens': 55, 'input_token_details': {}, 'output_token_details': {}})

In [19]:
with_message_history.invoke(
    {"ability": "math", "input": "What's its inverse"},
    config={"configurable": {"session_id": "foobar"}},
)

AIMessage(content='The inverse of cosine is called arccosine or cos^-1, representing the angle whose cosine is a given value.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 85, 'total_tokens': 110, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-90309ff0-de92-401f-b78a-396e4231a38b-0', usage_metadata={'input_tokens': 85, 'output_tokens': 25, 'total_tokens': 110, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})